# Creating a Chatbot linked to the Database

## 1. Importing libraries

In [5]:
%pip install surrealdb==1.0.3 google-genai==1.2.0 python-dotenv==1.0.1

Note: you may need to restart the kernel to use updated packages.


In [11]:
from surrealdb import Surreal
from dotenv import load_dotenv
from google import genai
from google.genai import types
import os
import json

load_dotenv(override=True)

True

## 2. Connect to the database

In [6]:
db = Surreal(os.getenv("SURREAL_URL"))
db.signin({"username": os.getenv("SURREAL_USER"), "password": os.getenv("SURREAL_PASS")})
db.use(os.getenv("SURREAL_NAMESPACE"), os.getenv("SURREAL_DATABASE"))

print(db.version())

surrealdb-2.2.0


## 4. Embed articles and save to database

In [16]:
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))
embed_model_name = "text-embedding-004"
llm_model_name = "gemini-2.0-flash"
sys_instruct = """
You are an Algerian legal expert AI trained to provide accurate answers.
You must strictly base your answer on the provided context.
You must avoid at all cost to mention in your answer that you are being provided with context.
You must strictly answer in Arabic.
Translate the keywords from context to Arabic.
Your response must adhere to any requested format in the question, if present.
Interpret the context to extract required information and present it directly, without mentioning the reasoning process.
If not, state clearly that you are unable to answer the question.
Do not fabricate or assume facts.
""".strip()

def search_articles(query):
    # more about this on https://ai.google.dev/gemini-api/docs/embeddings#python
    response = client.models.embed_content(
        model=embed_model_name,
        contents=query,
    )
    q_vector = response.embeddings[0].values

    results = db.query("""
        SELECT
            text,
            vector::similarity::cosine(embed_vector, $q_vector) AS score
        FROM articles
        ORDER BY score DESC
        LIMIT 5
        """,
        {"q_vector": q_vector}
    )

    print(json.dumps(results, indent=2, ensure_ascii=False), end="\n\n")
    return results

def ask_question(query):
    articles = search_articles(query)

    user_prompt = "```context\n"
    for article in articles:
        user_prompt += f"{article['text']}\n"
    user_prompt += "```\n"
    user_prompt += f"```question\n{query}\n```"

    print(user_prompt)
    print("\n------------------\n")

    # more about this on https://ai.google.dev/gemini-api/docs/text-generation?lang=python#system-instructions
    stream = client.models.generate_content_stream(
        model=llm_model_name,
        config=types.GenerateContentConfig(system_instruction=sys_instruct),
        contents=[user_prompt]
    )
    
    for chunk in stream:
        text_chunk = chunk.text
        if text_chunk is None:
            continue
        print(text_chunk, end="")

# question = "if I'm christian, what should I do in Algeria? and give me brief answer"
# question = "Anything about family law"
question = "معلومات عن الانتخابات"
ask_question(question)

[
  {
    "score": 0.852378955900973,
    "text": "المــادّة الأولى : الجزائر جمهوريّة ديمقراطيّة شعبيّة. وهي وحدة لا تتجزّأ."
  },
  {
    "score": 0.7908012383471339,
    "text": "المادة 2 : الإسلام  دين الدّولة."
  },
  {
    "score": 0.76106422425549,
    "text": "المادة 10 : لا يجوز للمؤسّسات أن تقوم بما يأتي : - الممارسات الإقطاعيّة، والجهويّة، والمحسوبيّة، - إقامة علاقات الاستغلال والتّبعيّة، - السّلوك المخالف للخُلُق الإسلاميّ وقيم ثورة نوفمبر."
  },
  {
    "score": 0.7561947051209077,
    "text": "المادة 8 : السّلطة التّأسيسيّة ملك للشّعب. يمارس الشّعب سيادته بواسطة المؤسّسات الدّستوريّة الّتي يختارها. يمارس الشّعب هذه السّيادة أيضا عن طريق الاستفتاء وبواسطة ممثّليه المنتخَبين. لرئيس الجمهوريّة أن يلتجئ إلى إرادة الشّعب مباشرة."
  },
  {
    "score": 0.7550015091041741,
    "text": "المادة 16 : الجماعات الإقليميّة للدّولة هي البلديّة والولاية. البلديّة هي الجماعة القاعديّة."
  }
]

```context
المــادّة الأولى : الجزائر جمهوريّة ديمقراطيّة شعبيّة. وهي وحدة لا تتجزّأ.
المادة 2 

### Useful links

- https://ai.google.dev/
- https://aistudio.google.com/
- https://ai.google.dev/gemini-api/docs/rate-limits